## CODE

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("open_api_key")

### 검색 증강 생성 개요

In [ ]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

vec1 = np.array([0,1,1,1])
vec2 = np.array([1,0,2,1])
vec3 = np.array([2,0,4,2])

print(f"벡터1과 벡터2의 유사도 : {cos_sim(vec1,vec2)}")
print(f"벡터2과 벡터3의 유사도 : {cos_sim(vec2,vec3)}")

In [ ]:
# OpenAI 임베딩 모델
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd

from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
query_result = embeddings.embed_query('저는 배가 고파요')
print(query_result)

In [ ]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 반등했어요',
]

df = pd.DataFrame(data,columns=['text'])
# print(df)

# 텍스트 -> 임베딩 벡터 변환 함수
def get_embedding(text):
    return embeddings.embed_query(text)

df['embeddings'] = df.apply(
    lambda row : get_embedding(row.text),
    axis=1
)
# print(df)

# 코사인 유사도 계산 함수
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))


def return_answer_candidate(df,query):
    query_embedding = get_embedding(query)

    df['similarity'] = df.embeddings.apply(lambda x : cos_sim(np.array(x), np.array(query_embedding)))

    top_three_doc = df.sort_values("similarity", ascending=False).head(3)

    return top_three_doc

sim_result = return_answer_candidate(df,'과일 값이 비싸다')
print(sim_result)


In [2]:
# 허깅페이스 제공 임베딩 모델
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd


# 텍스트 -> 임베딩 벡터 변환 함수
def get_embedding(text):
    return embeddings.embed_query(text)

# 코사인 유사도 계산 함수
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))


def return_answer_candidate(df,query):
    query_embedding = get_embedding(query)

    df['similarity'] = df.embeddings.apply(lambda x : cos_sim(np.array(x), np.array(query_embedding)))

    top_three_doc = df.sort_values("similarity", ascending=False).head(3)

    return top_three_doc

embeddings = SentenceTransformer('BAAI/bge-m3')
embeddings = HuggingFaceBgeEmbeddings(model_name = 'BAAI/bge-m3')
# embeddings = HuggingFaceEmbeddings(model_name = 'BAAI/bge-m3')

data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 반등했어요',
]

hugging_df = pd.DataFrame(data, columns=['text'])
hugging_df['embeddings'] = hugging_df['text'].apply(get_embedding)
# print(df)


sim_result = return_answer_candidate(hugging_df,'과일 값이 비싸다')
print(sim_result)


C:\Users\kimji\AppData\Local\Temp\ipykernel_39312\2227364096.py:30: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'BAAI/bge-m3')


                   text                                         embeddings  \
1           시장 물가가 올랐어요  [0.013636118732392788, 0.05754704773426056, -0...   
4  최근 비트코인 가격이 많이 반등했어요  [0.01619962975382805, 0.036948565393686295, -0...   
2  전통 시장에는 다양한 물품들을 팔아요  [0.01703060232102871, 0.04437505826354027, -0....   

   similarity  
1    0.702341  
4    0.673596  
2    0.667758  


### 문서로더

In [ ]:
import os
os.environ["USER_AGENT"] = "MyApp/1.0 (Custom Langchain Application)"

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/")

loader_multiple_pages = WebBaseLoader(
    ["https://python.langchain.com/docs/introduction/",
     "https://langchain-ai.github.io/langgraph"]
)

single_doc = loader.load()
print(single_doc[0].metadata)

docs = loader_multiple_pages.load()
print(docs[0].page_content)

실습 pdf 파일 다운 : https://www.kbfg.com/kbresearch/report/reportView.do?reportId=2000450

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader

loader = PyPDFLoader(r"pdf주소")
pages = loader.load_and_split()
print(f"청크의 수 : {len(pages)}")
print(pages[10])

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"pdf주소")
pages = loader.load_and_split()
print(f"청크의 수 : {len(pages)}")
print(pages[10])

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader

url = r"파일경로"

loader = PDFPlumberLoader(url)
pages = loader.load_and_split()
print(f"청크의 수 : {len(pages)}")
print(pages[10])

In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(url)
documents = loader.load()
print(f"청크의 수 : {len(pages)}")
print(documents[5])

In [ ]:
from langchain_community.document_loaders import UnstructuredCSVLoader

loader = UnstructuredCSVLoader(url)
documents = loader.load()
print(f"청크의 수 : {len(pages)}")
print(str(documents[0].metadata)[:500])
print(str(documents[0].page_content)[:500])

### 텍스트 분할

In [ ]:
# RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

url = r"파일경로"

loader = PyPDFLoader(url)
pages = loader.load()
print(f"총 글자 수 : {len(''.join([i.page_content for i in pages]))}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
texts = text_splitter.split_documents(pages) # 단순히 긴 문자열 (텍스트파일) 분할 : .split_text()
print(f"분할된 청크의 수 : {len(texts)}")

print(texts[1])
print(texts[1].page_content)
print(texts[2].page_content)
print(len(texts[1].page_content))
print(len(texts[2].page_content))

In [9]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(url)
pages = loader.load()

text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings(api_key=api_key))
chunks = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(chunks)}")


분할된 청크의 수 : 164


In [ ]:
print(chunks[3])
print(chunks[4])
print(chunks[5])

In [11]:
text_splitter = SemanticChunker(
	OpenAIEmbeddings(api_key=api_key),
	breakpoint_threshold_type="standard_deviation",
	breakpoint_threshold_amount=3,
)
chunks = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(chunks)}")

분할된 청크의 수 : 84


In [12]:
text_splitter = SemanticChunker(
	OpenAIEmbeddings(api_key=api_key),
	breakpoint_threshold_type="interquartile",
	breakpoint_threshold_amount=1.5,
)
chunks = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(chunks)}")

분할된 청크의 수 : 142


### VectorDB

In [ ]:
# Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

url = r"파일경로"

loader = PyPDFLoader(url)
pages = loader.load()
print(f"청크의 수 : {len(pages)}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(splits)}")

chunk_lengths = [len(chunk.page_content) for chunk in splits]
max_length = max(chunk_lengths)
min_length = min(chunk_lengths)
avg_length = sum(chunk_lengths)/len(chunk_lengths)

print(f"청크의 최대 길이 : {max_length}")
print(f"청크의 최소 길이 : {min_length}")
print(f"청크의 평균 길이 : {avg_length}")


embdiing_function = OpenAIEmbeddings(api_key=api_key)

persist_directory = r"VectorDB"
vector_store = Chroma.from_documents(documents=splits, embedding=embdiing_function, persist_directory=persist_directory ) # from_document를 반복해서 호출할 경우, 메모리가 중복으로 쌓일 수 있음.
print(f"문서의 수 : {vector_store._collection.count()}") # _collection.count() : Chroma DB에 적재된 문서 수 확인

vector_load = Chroma( embedding_function= embdiing_function, persist_directory=persist_directory)
print(f"문서의 수 : {vector_load._collection.count()}")


question = "수도권 주택 매매 전망"
top_three_docs = vector_load.similarity_search(question, k=2) # 상위 k개의 청크 검색
for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}")
    print(f"내용 : {doc.page_content[:150]}")
    print(f"메타데이터 : {doc.metadata}")
    print("--"*20)



In [ ]:
# FAISS
from langchain_community.vectorstores import FAISS

faiss_db = FAISS.from_documents(documents=splits, embedding=embdiing_function)
print(f"문서의 수 : {faiss_db.index.ntotal}")

faiss_directory = r"FAISS"
faiss_db.save_local(faiss_directory)

new_db_faiss = FAISS.load_local(faiss_directory, embdiing_function , allow_dangerous_deserialization=True )

question = "수도권 주택 매매 전망"
docs = new_db_faiss.similarity_search(question)
for i, doc in enumerate(docs, 1):
    print(f"문서 {i}")
    print(f"내용 : {doc.page_content[:150]}")
    print(f"메타데이터 : {doc.metadata}")
    print("--"*20)



문서의 수 : 135
문서 1
내용 : 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가
메타데이터 : {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': 'C:\\Users\\kimji\\Desktop\\ProgramFile\\Study\\2024 KB 부동산 보고서_최종.pdf', 'total_pages': 84, 'page': 14, 'page_label': '15'}
----------------------------------------
문서 2
내용 : 18 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
그림Ⅰ-30. 수도권 입주물량과 전세가격 변동률 추이  그림Ⅰ-31. 기타지방 입주물량과 전세가격 변동률 추이 
 
 
 
자료: KB국민은행, 부동산114  자료: KB국민은행, 부동산114
메타데이터 : {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': 'C:\\Users\\kimji\\Desktop\\ProgramFile\\Study\\2024 KB 부동산 보고

### RAG 실습

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

url = r"파일경로"

# 인덱싱 과정
# 1) 문서 로드 및 텍스트 분할
loader = PyMuPDFLoader(url)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"분할된 청크 수 : {len(chunks)}")

# 2) 임베딩 생성과 DB 적재,관리
embdiing_function = OpenAIEmbeddings(api_key=api_key)
persist_directory = r"Study_LLM\VectorDB"
vector_store = Chroma.from_documents(
    documents = chunks,
    embedding=embdiing_function,
    persist_directory=persist_directory,
)
print(f"문서의 수 : {vector_store._collection.count}")


# 쿼리 과정
# 1) 검색 및 재정렬
retriever = vector_store.as_retriever(search_kwards={"k":3})

# 2) 프롬프트 템플릿 설정
template = """ 당신은 KB 부동산 보고서 전문가 입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요. 컨텍스트 ; {context} """
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("placeholder", "{chat_history}"), # 대화 기록용, 이전 대화 내용을 삽입.
        ("human", "{qustion}")
    ]
)

model = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0, api_key=api_key)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    RunnablePassthrough.assing(
        context = lambda x : format_docs(retriever.invoke(x["question"]))
    )
    | prompt
    | model
    | StrOutputParser()
)

# 메모리 설정 및 챗봇 실행
chat_history = ChatMessageHistory()
chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id : chat_history,
    input_messages_key="question",
    history_messages_key = "chat_history",
)



def chat_with_bot():
    session_id = "user_session",
    print("KB 부동산 챗봇 / 종료는 \"quit\"입력")
    while True:
        user_input = input("사용자 : ")
        if user_input.lower() == "quit":
            break
        response = chain_with_memory.invoke(
            {"question" : user_input},
            {"configurable" : {"session_id" : session_id}}
        )

        print(f"챗봇 : {response}")


In [ ]:
!ngrok config add-authtoken <본인 코드>

'ngrok'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
from pyngrok import ngrok

ngrok.kill() 

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(f"앱 접속 url : {public_url}")

!streamlit run app.py